# Analyzing Feb 2019 16s Sequencing Data
## Written by Kathryn Cogert, Feb 5th, 2019

### Background:
USEARCH was used to clean up this sequencing data collected from several samples.  Namely-- 

Mini column samples:
 - KC AMX D46 (Day 46 granules from large anammox column reactor, taken just before mini column inoculation)
 - KC AC Minicolumn Upper (Day 23 granules taken from minicolumn. Red granules stripped of AC only)
 - KC AC Minicolumn Mixed (Day 23 granules taken from minicolumn. mixture of black and red granules)
 - KC AC Batch (Granules from the failed batch experiment)

Red & Grey Anammox Granules:
 - KC AMX D46 (Day 46 granules from large anammox column reactor, taken just before mini column inoculation)
 - KC Red (Red granules taken from large anammox column reactor)
 - KC Grey (Grey granules taken from large anammox column reactor)

### Objective:
This analysis is intended to drop sequencing results into a CSV for data visualization in R. Preprocessing of data was done with USEARCH - see in [Pipeline.ipynb](Pipeline.ipynb))



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import ipywidgets as widgets
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,7)

### Step 1) Load in OTU counts per sample table

In [4]:
otu_count = pd.read_csv('zotutab.txt', sep='\t').rename(columns={'#OTU ID':'OTU #'})
otu_count.head()

,OTU #,AMX.MiniCol1.D0,AMX.MiniCol2.D0.AC,AMX.MiniCol3.D0.AC,AMX.MiniCol1.D31,AMX.MiniCol3.D31.AC,AMX.MiniCol2.D31.AC,KC.AC.MiniColumn.Upper,KC.AMX.Grey,KC.AMX.D46,...,KC3.C2.D118.R1,KC4.C2.D118.R2,KC7.C1.D0.R2,KC10.C1.D31.R2,KC11.C2.D31.R2,KC1.C1.D118.R2b,KC5.C3.D118.R1,KC8.C2.D0.R2,KC1.C1.D118.R2,KC9.C3.D0.R2
0,Otu2,3954,3234,6117,2706,4806,5677,2655,4655,6926,...,7057,5614,9420,10954,7060,6443,5219,15287,3793,13436
1,Otu7,2485,676,1029,308,448,668,568,592,1520,...,2680,1347,2027,8606,10897,2609,963,4952,1090,4806
2,Otu12,1648,504,731,301,483,661,822,252,364,...,2585,1331,1616,5961,3425,2086,1856,2473,1021,2347
3,Otu78,67,137,83,252,404,53,136,16,4,...,118,331,240,106,195,346,133,128,84,85
4,Otu8,1006,1813,749,1808,1125,668,3803,527,2424,...,1715,2440,1110,506,2397,5263,4335,584,2588,509


### Step 2) Load in the OTU taxonomy assignment table & data clean

In [5]:
otu_id = pd.read_table('otus.sintax', header=None, delimiter='\t', index_col=None)
otu_id_clean = otu_id. \
join(otu_id[3].str.split(',', 5, expand=True). \
rename(columns={0:'Domain',1:'Phylum',2:'Class',3:'Order',4:'Family',5:'Genus'})). \
drop(columns=[1,2,3]). \
rename(columns={0:'OTU #'})
otu_id_clean.iloc[:,1:] = otu_id_clean.iloc[:,1:].apply(lambda x: x.str.replace('[a-z]:',''))

otu_id_clean.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,OTU #,Domain,Phylum,Class,Order,Family,Genus
0,Otu1,NaN,NaN,NaN,NaN,NaN,NaN
1,Otu3,Bacteria,"""Bacteroidetes""",Cytophagia,Cytophagales,None,None
2,Otu4,Bacteria,"""Planctomycetes""",Planctomycetia,Candidatus_Brocadiales,Candidatus_Brocadiaceae,Candidatus_Brocadia
3,Otu2,Bacteria,"""Proteobacteria""",Betaproteobacteria,Rhodocyclales,Rhodocyclaceae,None
4,Otu5,Bacteria,Nitrospirae,"""Nitrospira""","""Nitrospirales""","""Nitrospiraceae""",Nitrospira


### Step 3) Merge two dataframes together and normalize columns

In [6]:
df = otu_id_clean.merge(right=otu_count, on='OTU #')
colsum = pd.DataFrame(df.sum(axis=0)[1:7]).transpose()
df.iloc[:,7:] = df.iloc[:,7:] /  df.iloc[:,7:].sum()
df.head()

,OTU #,Domain,Phylum,Class,Order,Family,Genus,AMX.MiniCol1.D0,AMX.MiniCol2.D0.AC,AMX.MiniCol3.D0.AC,...,KC3.C2.D118.R1,KC4.C2.D118.R2,KC7.C1.D0.R2,KC10.C1.D31.R2,KC11.C2.D31.R2,KC1.C1.D118.R2b,KC5.C3.D118.R1,KC8.C2.D0.R2,KC1.C1.D118.R2,KC9.C3.D0.R2
0,Otu1,NaN,NaN,NaN,NaN,NaN,NaN,0.008426,0.059842,0.040029,...,0.182731,0.226960,0.133017,0.035956,0.101534,0.169109,0.141604,0.037004,0.216278,0.088960
1,Otu3,Bacteria,"""Bacteroidetes""",Cytophagia,Cytophagales,None,None,0.034887,0.070238,0.058554,...,0.049214,0.069502,0.036131,0.020355,0.018887,0.085447,0.055465,0.022088,0.076392,0.019310
2,Otu4,Bacteria,"""Planctomycetes""",Planctomycetia,Candidatus_Brocadiales,Candidatus_Brocadiaceae,Candidatus_Brocadia,0.046446,0.126420,0.031029,...,0.046805,0.037284,0.052610,0.015002,0.033943,0.035222,0.033507,0.047279,0.031638,0.046271
3,Otu2,Bacteria,"""Proteobacteria""",Betaproteobacteria,Rhodocyclales,Rhodocyclaceae,None,0.075545,0.067644,0.118404,...,0.056482,0.045140,0.081191,0.097877,0.053252,0.031061,0.045766,0.126466,0.034523,0.131103
4,Otu5,Bacteria,Nitrospirae,"""Nitrospira""","""Nitrospirales""","""Nitrospiraceae""",Nitrospira,0.000764,0.014955,0.001471,...,0.073305,0.082681,0.015368,0.002368,0.008591,0.094997,0.106177,0.001795,0.097472,0.001912


### Step 4) Note unidentified organisms as such

In [24]:
df.fillna('Unidentified', inplace=True)
df.head()


,OTU #,Domain,Phylum,Class,Order,Family,Genus,AMX.MiniCol1.D0,AMX.MiniCol2.D0.AC,AMX.MiniCol3.D0.AC,...,KC3.C2.D118.R1,KC4.C2.D118.R2,KC7.C1.D0.R2,KC10.C1.D31.R2,KC11.C2.D31.R2,KC1.C1.D118.R2b,KC5.C3.D118.R1,KC8.C2.D0.R2,KC1.C1.D118.R2,KC9.C3.D0.R2
0,Otu1,Unidentified,Unidentified,Unidentified,Unidentified,Unidentified,Unidentified,0.008426,0.059842,0.040029,...,0.182731,0.226960,0.133017,0.035956,0.101534,0.169109,0.141604,0.037004,0.216278,0.088960
1,Otu3,Bacteria,Bacteroidetes,Bacteroidia,Cytophagales,Microscillaceae,uncultured,0.034887,0.070238,0.058554,...,0.049214,0.069502,0.036131,0.020355,0.018887,0.085447,0.055465,0.022088,0.076392,0.019310
2,Otu2,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Rhodocyclaceae,Denitratisoma,0.075545,0.067644,0.118404,...,0.056482,0.045140,0.081191,0.097877,0.053252,0.031061,0.045766,0.126466,0.034523,0.131103
3,Otu4,Bacteria,Planctomycetes,Brocadiae,Brocadiales,Scalinduaceae,Candidatus_Scalindua,0.046446,0.126420,0.031029,...,0.046805,0.037284,0.052610,0.015002,0.033943,0.035222,0.033507,0.047279,0.031638,0.046271
4,Otu6,Bacteria,Bacteroidetes,Ignavibacteria,SJA28,Unidentified,Unidentified,0.047822,0.029346,0.042043,...,0.096956,0.020415,0.017074,0.056757,0.032185,0.039643,0.040172,0.050158,0.038301,0.054077


### Step 5) Save as CSV

In [25]:
df.to_csv('AC_Column_Abundances.csv')

### Supplemental

Previously used interactive plot to visualize by taxonomic rank.

In [113]:
@interact(Rank=['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus'],
         Experiment = ['Activated Carbon', 'Granule Color'])
def compare_v(Rank='Order', Experiment='Activated Carbon'): 
    #to_plot = (df.groupby([Rank]).sum()/ df. \
    #           groupby([Rank]).sum().sum()). \
    #reindex(sorted(df.columns), axis=1)
    to_plot = df.groupby([Rank]).sum().sort_values('KC.AMX.D46', 
                                                   ascending=False).transpose()
    if Experiment=='Activated Carbon':
        to_plot = to_plot.loc[['KC.AMX.D46',
                               'KC.AC.MiniColumn.Upper',
                               'KC.AC.MiniColumn.Mixed', 
                               'KC.AC.Batch']]
    elif Experiment == 'Granule Color':
        to_plot = to_plot.loc[['KC.AMX.D46',
                               'KC.AMX.Grey', 
                               'KC.AMX.Red']]
    fig=to_plot.plot(kind='bar', stacked=True)
    
    #for i in [0,2,4,6,8, 10]: 
    # Omit KC.AMXRT results b/c not in v10 results.
    #    fig=to_plot.iloc[i:i+2,:].plot(kind='bar', stacked=True)
    plt.title(Experiment, color='white', fontsize=14, fontweight='bold')
    plt.xticks(color='white', size=16, rotation=45)
    plt.yticks(np.arange(0,1.1,0.1), color='white')
    fig.legend(bbox_to_anchor=(0.75, -0.1), ncol=2)



interactive(children=(Dropdown(description='Rank', index=3, options=('Domain', 'Phylum', 'Class', 'Order', 'Fa…